##  This project is an AI-powered chatbox assistant with gradio that interacts with a SQLite database to manage ticket prices. 

Users can check the price of tickets to popular cities such as Paris, London, Tokyo, and Sydney. 

The assistant also allows users to update the prices of existing cities directly through chat, users can also add new cities or change ticket prices on the go.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [11]:

# There's a particular dictionary structure that's required to describe our function:
set_ticket_price_function = {
    "name": "set_ticket_price",
    "description": "Set, update or change the ticke price for a destination city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city whose ticket price should be updated",
            },
            "price": {
                "type": "number",
                "description": "The new price of the ticket"
            }
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}


price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}



In [12]:
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": set_ticket_price_function}]


#create database for prices of tickets and insert data for 4 cities


In [20]:
DB = "prices.db"

ticket_prices = {
    "london": 799,
    "paris": 899,
    "tokyo": 1420,
    "sydney": 2999
}

# Connect to the database (creates it if it doesn't exist)
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    
    # Create the table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS prices (
            city TEXT PRIMARY KEY,
            price REAL
        )
    ''')
    
    # Insert data 
    for city, price in ticket_prices.items():
        cursor.execute('''
            INSERT INTO prices (city, price)
            VALUES (?, ?)
            ON CONFLICT(city) DO UPDATE SET price=excluded.price
        ''', (city.lower(), price))
    
    # Commit changes
    conn.commit()
    

In [21]:
def handle_tool_calls(message):
    responses = []

    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        result = "Unknown tool or error"


        if function_name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')

            if not city:
                result = "No city provided"
            else:
                result = get_ticket_price(city)
        
        elif function_name == "set_ticket_price":
            city = arguments.get('destination_city')
            price = arguments.get('price')

            if not city or price is None:
                result = "No city or price provided"
            else:
                try:
                    update_ticket_price(city, price)
                    result = f"Ticket price to {city} set to ${price}"
                except ValueError:
                    result = "Price must be a number."
        else:
            result = "Unknown function"
        
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })

    return responses

In [22]:
def update_ticket_price(city, price):
    city = city.strip().lower()
    print(f"DATABASE TOOL CALLED: Updating price for {city}", flush=True)

    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [16]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [23]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for Abuja
DATABASE TOOL CALLED: Updating price for abuja
DATABASE TOOL CALLED: Getting price for Abuja
DATABASE TOOL CALLED: Updating price for lagos
